In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)
import scipy.ndimage as ndimage
import time

In [ ]:

def RotateImage(img, angle,cval=255):
    img = img.copy()
    img = ndimage.rotate(img, angle, cval=cval)
    return img

def get_SIFT_Rotation_angle(img_1, img_2):
    img_1 = img_1.copy()
    img_2 = img_2.copy()
    
    sift = cv2.SIFT_create()
    kp_1 = sift.detect(img_1,None)
    kp_2 = sift.detect(img_2,None)
    
    start = time.time()
    des_1 = sift.compute(img_1,kp_1)[1] 
    des_2 = sift.compute(img_2,kp_2)[1]
    end = time.time() - start
    
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des_1,des_2, k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.3 * n.distance:
            good.append([m]) 
            

    src_pts = np.float32([ kp_1[m[0].queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp_2[m[0].trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    img3 = cv2.drawMatchesKnn(img_1,kp_1,img_2,kp_2,good,None,    matchColor = (200,30,30), singlePointColor = None,flags = 2)
    return np.rad2deg(np.arctan2(M[1,0],M[0,0])) , end,img3

In [ ]:
def get_FREAK_Rotation_angle(img_1, img_2):
    img_1 = img_1.copy()
    img_2 = img_2.copy()
    
    sift = cv2.SIFT_create()
    kp_1 = sift.detect(img_1,None)
    kp_2 = sift.detect(img_2,None)
    
    start = time.time()
    freakExtractor = cv2.xfeatures2d.FREAK_create()
    kp_1,des_1 = freakExtractor.compute(img_1,kp_1)
    kp_2,des_2 = freakExtractor.compute(img_2,kp_2)
    end = time.time() - start
    
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des_1,des_2, k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.4 * n.distance:
            good.append([m])   
    
    src_pts = np.float32([ kp_1[m[0].queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp_2[m[0].trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    img3 = cv2.drawMatchesKnn(img_1,kp_1,img_2,kp_2,good,None,    matchColor = (200,30,30), singlePointColor = None,flags = 2)
    return np.rad2deg(np.arctan2(M[1,0],M[0,0])),end, img3


In [ ]:
def get_ORB_Rotation_angle(img_1,img_2):
    img_1 = img_1.copy()
    img_2 = img_2.copy()
    
    orb = cv2.ORB_create()
    kp_1 = orb.detect(img_1,None)
    kp_2 = orb.detect(img_2,None)
    
    start = time.time()
    kp_1,des_1 = orb.compute(img_1,kp_1)
    kp_2,des_2 = orb.compute(img_2,kp_2)
    end = time.time() - start
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des_1,des_2)
    matches = sorted(matches, key = lambda x:x.distance)
    good = matches[:int(len(matches)*0.2)]
    
    src_pts = np.float32([ kp_1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp_2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    img3 = cv2.drawMatches(img_1,kp_1,img_2,kp_2,good,None,    matchColor = (200,30,30), singlePointColor = None,flags = 2)
    return np.rad2deg(np.arctan2(M[1,0],M[0,0])),end, img3


In [ ]:
def report(img_1,img_2,img_res,angle_applied,angle_detected,_time,method="SIFT"):
    img_rerotate = RotateImage(img_2, angle=angle_detected, cval=128)
    angle_detected = round(angle_detected,4)
    _time = round(_time,4)
    angle_applied = round(angle_applied,4)
    w,h = 2,2
    I = 1 
    plt.subplot(w,h,I);I+=1;plt.imshow(img_1, 'gray');plt.title("Original Image");plt.axis('off')
    plt.subplot(w,h,I);I+=1;plt.imshow(img_2, 'gray');plt.title("Rotated Image | Applied angle: {0}".format(angle_applied));plt.axis('off')
    plt.subplot(w,h,I);I+=1;plt.imshow(img_res, 'gray');plt.title("{2} Matches | Comput Time: {0}s | Detected angle: {1}".format(_time,angle_detected,method));plt.axis('off')
    plt.subplot(w,h,I);I+=1;plt.imshow(img_rerotate, 'gray');plt.title("Re-Rotated Image");plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
def get_three_random_angles():
    return np.random.randint(0,360,3)

img_1 = cv2.imread("test.jpg")[:,:,::-1]

angles = get_three_random_angles()
for rotate_angle in angles:
    img_2 = RotateImage(img_1, rotate_angle,50)
    angle,end,img_res = get_FREAK_Rotation_angle(img_1, img_2)
    img_3 = RotateImage(img_2, angle=angle, cval=128)
    angle,end
    report(img_1,img_2,img_res,rotate_angle,angle,end,"FREAK")

    img_2 = RotateImage(img_1, rotate_angle,50)
    angle,end,img_res = get_SIFT_Rotation_angle(img_1, img_2)
    img_3 = RotateImage(img_2, angle=angle, cval=128)
    angle,end
    report(img_1,img_2,img_res,rotate_angle,angle,end,"SIFT")

    img_2 = RotateImage(img_1, rotate_angle,50)
    angle,end,img_res = get_ORB_Rotation_angle(img_1, img_2)
    img_3 = RotateImage(img_2, angle=angle, cval=128)
    angle,end
    report(img_1,img_2,img_res,rotate_angle,angle,end,"ORB")